In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import imageio
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers

In [3]:
df = pd.read_csv('/content/drive/My Drive/LeWagon_ODR/full_df_cleaned_v3.csv')

### **New Dataframe avec uniquement les observations Normal & Cataracte**

In [4]:
df_n = df[df['tarstr']=='N']

In [5]:
df_c = df[df['tarstr']=='C']

In [6]:
df_N_C = pd.concat([df_n, df_c], ignore_index=True)

**New Columns with the image PATH**

In [7]:
IMAGE_PATH = '/content/drive/My Drive/LeWagon_ODR/preprocessed_images2/'

In [8]:
df_N_C['filepath'] = IMAGE_PATH + df_N_C['filename']

**Add every image to the img_data array**

In [9]:
img_data = []
number_id_nofile = []

for i in range(len(df_N_C)):
  try:
    img_data.append(imageio.imread(df_N_C['filepath'][i]))
  except:
    number_id_nofile.append(df_N_C.index[i])

In [10]:
img_data_array = np.array(img_data)

**Create our X and y**

In [11]:
X = img_data_array

In [12]:
y = df_N_C['C']

**Train Test Split (en stratifiant sur y)**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

**Normalize our images**

In [14]:
X_train = X_train / 255
X_test = X_test / 255

**Baseline model**

In [20]:
def initialize_model():
  model = models.Sequential()

  model.add(layers.Conv2D(32, (3,3), input_shape=(256, 256, 3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(2,2)))
  model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(3,3)))
  model.add(layers.Conv2D(256, (3,3), activation='relu', padding='same'))
  model.add(layers.MaxPool2D(pool_size=(3,3)))
  ### Flattening
  model.add(layers.Flatten())
  ### One fully connected
  model.add(layers.Dense(120, activation='relu'))
  model.add(layers.Dropout(rate=0.5))
  model.add(layers.Dense(60, activation='relu'))
  model.add(layers.Dropout(rate=0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', 
        optimizer='adam',
        metrics=['accuracy'])

  return model

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 256)       2

In [21]:
model = initialize_model()
es = EarlyStopping(patience=20, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    epochs=100,
                    batch_size=16, 
                    verbose=1,
                    callbacks=[es])

Epoch 1/100
94/94 [==============================] - 38s 49ms/step - loss: 0.5130 - accuracy: 0.8158 - val_loss: 0.2879 - val_accuracy: 0.9203
Epoch 2/100
94/94 [==============================] - 3s 36ms/step - loss: 0.3269 - accuracy: 0.9194 - val_loss: 0.2790 - val_accuracy: 0.9203
Epoch 3/100
94/94 [==============================] - 3s 36ms/step - loss: 0.3227 - accuracy: 0.9208 - val_loss: 0.2727 - val_accuracy: 0.9203
Epoch 4/100
94/94 [==============================] - 3s 35ms/step - loss: 0.3026 - accuracy: 0.9207 - val_loss: 0.2796 - val_accuracy: 0.9203
Epoch 5/100
94/94 [==============================] - 3s 36ms/step - loss: 0.3284 - accuracy: 0.8944 - val_loss: 0.3180 - val_accuracy: 0.9203
Epoch 6/100
94/94 [==============================] - 3s 36ms/step - loss: 0.3572 - accuracy: 0.9093 - val_loss: 0.2683 - val_accuracy: 0.9203
Epoch 7/100
94/94 [==============================] - 3s 35ms/step - loss: 0.3526 - accuracy: 0.9037 - val_loss: 0.2775 - val_accuracy: 0.9203
Epoch

**Evaluate on X_test & y_test**

In [22]:
model.evaluate(X_test, y_test, verbose=0)

[0.24723882973194122, 0.9529540538787842]

**Save the Baseline model**

In [23]:
import joblib

In [24]:
filename = '/content/drive/My Drive/LeWagon_ODR/baseline_model_N_C.joblib'
joblib.dump(model, filename)

TypeError: ignored